## About this demo
This demo describes step-by-step how to use BMF to develop a transcoding program, including video transcoding, audio transcoding, and image transcoding. In it, you can familiarize yourself with how to use BMF and how to use FFmpeg-compatible options to achieve the capabilities you need.

## 1. Environmental preparation

### 1.1 FFmpeg
FFmpeg 4.x or 5.x is needed by BMF when transcoding, check versions via apt:

In [ ]:
! apt show ffmpeg | grep "^Package:\|^Version:"

If the version meets the requirements, install ffmpeg via apt:

In [ ]:
! apt install -y ffmpeg

Otherwise, you need to compile ffmpeg from source, you can use the script we provided(only linux and macos now):

In [ ]:
! git clone https://github.com/BabitMF/bmf bmf
! ./bmf/scripts/build_ffmpeg.sh nasm yasm x264 x265

### 1.2 BMF
BMF can be installed in many ways, we use pip here:

In [ ]:
! pip3 install BabitMF

### 1.3 wurlitzer(optional)
This package is installed to show the BMF C++ logs in the colab console, otherwise only python logs are printed. This step is not necessary if you're not in a Colab or iPython notebook environment.

In [ ]:
!pip install wurlitzer
%load_ext wurlitzer

## 2. Transcode demo

Download the video file we will be using first:

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1l8bDSrWn6643aDhyaocVStXdoUbVC3o2/view?usp=sharing -O big_bunny_10s_30fps.mp4

In [ ]:
! ffprobe big_bunny_10s_30fps.mp4

### 2.1 video transcoding

#### 2.1.1 remuxing demo
This demo shows how to demux a video in mp4 format and remuxing it into hls slices without involving audio and video decoding and encoding:


In [ ]:
import bmf

input_video_path = "./big_bunny_10s_30fps.mp4"
output_path = "./remux_output.m3u8"

# create graph
graph = bmf.graph()

# decode
video = graph.decode({
    "input_path": input_video_path,
    "video_codec": "copy",
    "audio_codec": "copy"
})

(
    bmf.encode(
        video['video'],
        video['audio'],
        {
            "output_path": output_path,
            "format": "hls",
            "mux_params": {
                "hls_list_size": "0",
                "hls_time": "2",
                "hls_segment_filename": "./file%03d.ts"
            }
        }
    ).run()
)

In [ ]:
! ffprobe remux_output.m3u8
! rm -rf remux_output.m3u8 file*.ts

#### 2.1.2 decoding and encoding demo
This demo shows how to decode a H.264 video, drop the audio, scale the video to 720x576p resolution, and encode it with x265 encoder. You can even do it in one line of code:

In [ ]:
import bmf

input_video_path = "./big_bunny_10s_30fps.mp4"
output_path = "./decode_scale_encode_output.mp4"
(
    bmf.graph()
        .decode({'input_path': input_video_path})['video']
        .scale(720, 576)
        .encode(None, {
            "output_path": output_path,
            "video_params": {
                "codec": "libx265"
            }
        }).run()
)

In [ ]:
! ffprobe decode_scale_encode_output.mp4
! rm -rf decode_scale_encode_output.mp4

#### 2.1.3 multi stream demo
This demo shows how to decode the original video, use the ffmpeg filter capability, and encode multi videos with different options:

In [ ]:
import bmf

input_video_path = "./big_bunny_10s_30fps.mp4"
output_path0 = "./decode_encode_multi_output0.mp4"
output_path1 = "./decode_encode_multi_output1.mp4"

streams = bmf.graph().decode({'input_path': input_video_path})
split_streams = streams['video'].split()
bmf.encode(split_streams[0], streams['audio'], {
    "output_path": output_path0,
    "video_params": {
        "codec": "libx264",
        "x264-params": "ssim=1:psnr=1"
    }
})
(
    bmf.encode(split_streams[1], streams['audio'], {
            "output_path": output_path1,
            "video_params": {
                "codec": "libx265",
                "preset": "fast",
                "crf": "23"
            }
        }).run()
)

In [ ]:
! ffprobe decode_encode_multi_output0.mp4
! echo "----------------------------------------------------------------------"
! ffprobe decode_encode_multi_output1.mp4

! rm -rf decode_encode_multi_output0.mp4 decode_encode_multi_output1.mp4

### 2.2 audio transcoding


This demo shows how to use BMF to decode the input video, extract the audio part, add a piece of null audio before and after, and then encode the output wav file:

In [ ]:
import bmf

input_video_path = "./big_bunny_10s_30fps.mp4"
output_path = "./with_null_audio.wav"

# create graph
graph = bmf.graph()

# decode
streams = graph.decode({
    "input_path": input_video_path
})

# create a null audio stream
audio_stream1 = graph.anullsrc('r=48000', 'cl=2').atrim('start=0', 'end=6')
audio_stream2 = graph.anullsrc('r=48000', 'cl=2').atrim('start=0', 'end=6')
concat_audio = (
    bmf.concat(audio_stream1, streams['audio'], audio_stream2, n=3, v=0, a=1)
)

(
    bmf.encode(
        None,
        concat_audio,
        {
            "output_path": output_path,
            "audio_params": {
                "codec": "aac",
                "bit_rate": 128000,
                "sample_rate": 44100,
                "channels": 2
            }
        }
    )
    .run()
)

In [ ]:
! ffprobe with_null_audio.wav
! rm -rf with_null_audio.wav

### 2.3 image transcoding
This demo shows how to decode a video, detect and skip black frames, encode and output to the pipeline, and then write files at the application layer. You can also do other custom operations, such as uploading directly to the cloud to avoid disk IO:

In [ ]:
import bmf

input_video_path = "./big_bunny_10s_30fps.mp4"

graph = bmf.graph()
streams = graph.decode({
    "input_path": input_video_path,
})
video = streams['video'].ff_filter("blackframe", threshold=32).ff_filter("metadata", "select:key=lavfi.blackframe.pblack:value=100:function=less")
vframes_num = 2
result = (
    bmf.encode(
        video,
        None,
        {
            "push_output": 1,
            "vframes": vframes_num,
            "format": "image2pipe",
            "avio_buffer_size": 65536, #16*4096
            "video_params": {
                "codec": "jpg",
                "width": 640,
                "height": 480
            },
        }
    )
    .start()
)
write_num = 0
for i, packet in enumerate(result):
    avpacket = packet.get(bmf.BMFAVPacket)
    data = avpacket.data.numpy()
    if write_num < vframes_num:
        output_path = "./simple_image" + str(write_num)+ ".jpg"
        write_num = write_num + 1
        with open(output_path, "wb") as f:
            f.write(data)

In [ ]:
! ffprobe simple_image0.jpg
! rm -rf simple_image*.jpg